# Improvement Trial Using Previous Model, backtracking temperature and increasing discount

This trial will test allowing an okay AI to take riskier moves. I will work on viz code in the meantime. Later I will test the effect of increasing the lag of the q-score change. This should be safer, as the AI are unlikely to choose as many tickets as they previously did.

In [1]:
import gpu_rec
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 0.01

init_game = Game(n_players=5, q_lag=1, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []

for i, player in enumerate(player_list):
    player.ai.load_models('ai_h5/p%d_test09_%%s.h5' % i)



In [3]:
def which_min(x):
    min_val = min(x)
    return x.index(min_val)

def which_max(x):
    max_val = max(x)
    return x.index(max_val)

def aid_weakest_ai(players, wins):
    players[which_min(wins)].ai.receive_models_from_other_ai(players[which_max(wins)].ai)
    

In [4]:
temperatures = [ 0.1, 0.05, 0.01, 0.01, 0.001, 0.00001, 0.00001, 0., 0., 0.]
n_trials_per_cycle = 150
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print '---TEMPERATURE: %.03f---' % temperature
    print '+IDX=%d+' % temp_index
    test_config['discount'] = (2+temp_index)/11.
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    aid_weakest_ai(player_list[1:], player_wins[1:])
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

---TEMPERATURE: 0.100---
+IDX=0+
in cycle 0
......................................................................................................................................................in cycle 1
......................................................................................................................................................in cycle 2
......................................................................................................................................................WINNING SCORE AVERAGE: 40.2
NUMBER OF TURNS AVERAGE: 234.0
---PLAYER 0 STATS---
WIN AVERAGE: 0.000
SCORE AVERAGE: -33.6
---PLAYER 1 STATS---
WIN AVERAGE: 0.260
SCORE AVERAGE: 22.3
---PLAYER 2 STATS---
WIN AVERAGE: 0.291
SCORE AVERAGE: 23.4
---PLAYER 3 STATS---
WIN AVERAGE: 0.222
SCORE AVERAGE: 20.7
---PLAYER 4 STATS---
WIN AVERAGE: 0.240
SCORE AVERAGE: 21.1
---TEMPERATURE: 0.050---
+IDX=1+
in cycle 0
................................................................................

ai.py:414: RuntimeWarning: invalid value encountered in divide
  return (selected)/np.sum(selected)


...................................

ai.py:386: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


...........................................................................................................in cycle 1
......................................................................................................................................................in cycle 2
......................................................................................................................................................WINNING SCORE AVERAGE: 44.8
NUMBER OF TURNS AVERAGE: 226.0
---PLAYER 0 STATS---
WIN AVERAGE: 0.000
SCORE AVERAGE: -23.8
---PLAYER 1 STATS---
WIN AVERAGE: 0.273
SCORE AVERAGE: 28.0
---PLAYER 2 STATS---
WIN AVERAGE: 0.253
SCORE AVERAGE: 27.0
---PLAYER 3 STATS---
WIN AVERAGE: 0.211
SCORE AVERAGE: 24.9
---PLAYER 4 STATS---
WIN AVERAGE: 0.300
SCORE AVERAGE: 28.2
---TEMPERATURE: 0.010---
+IDX=3+
in cycle 0
......................................................................................................................................................in cycle 1
......

ai.py:413: RuntimeWarning: overflow encountered in power
  selected = x**(1./(temperature)**0.5)


.....................................................................................................................................................in cycle 1
......................................................................................................................................................in cycle 2
......................................................................................................................................................WINNING SCORE AVERAGE: 14.0
NUMBER OF TURNS AVERAGE: 214.8
---PLAYER 0 STATS---
WIN AVERAGE: 0.058
SCORE AVERAGE: -20.0
---PLAYER 1 STATS---
WIN AVERAGE: 0.236
SCORE AVERAGE: -3.2
---PLAYER 2 STATS---
WIN AVERAGE: 0.231
SCORE AVERAGE: -2.6
---PLAYER 3 STATS---
WIN AVERAGE: 0.280
SCORE AVERAGE: -1.4
---PLAYER 4 STATS---
WIN AVERAGE: 0.231
SCORE AVERAGE: -3.3
---TEMPERATURE: 0.000---
+IDX=6+
in cycle 0
.............................................................................................................................

KeyboardInterrupt: 

in the discount = 7/11 iteration, moving from 0.001 to 0.000 temperature, the score worsened.

I will try limiting the discount to around 0.5, and also try increasing the q-lag so that rewards can be slightly delayed

In [ ]:
print game.turn
print i
print cycle
print temp_index
for player in players:
    print player
    print len(player.determine_possible_tracks())

In [ ]:
np.mean(overall_winning_scores,1)

In [ ]:
np.mean(overall_player_scores, ( 2,))

In [5]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test10_%%s.h5' % i)

In [ ]:
for player in player_list:
    print player